In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Файл(ы) "{name}" размером {length} байт загружен(ы)'.format(
      name=fn, length=len(uploaded[fn])))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return " ".join([w for w in text.split() if w not in stop_words])

train["comment_text"] = train["comment_text"].astype(str).apply(clean_text)
test["comment_text"] = test["comment_text"].astype(str).apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
max_words = 100000
max_len = 150

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train["comment_text"])

X = tokenizer.texts_to_sequences(train["comment_text"])
X = pad_sequences(X, maxlen=max_len, padding="post", truncating="post")

label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[label_cols].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [2]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Sequential

embedding_dim = 128

model = Sequential([
    Input(shape=(max_len,)),
    Embedding(input_dim=max_words, output_dim=embedding_dim),
    LSTM(128, return_sequences=True),
    Dropout(0.4),
    BatchNormalization(),
    LSTM(64, return_sequences=False),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(label_cols), activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

model.summary()

KeyboardInterrupt: 

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss', patience=2, restore_best_weights=True
)
checkpoint = ModelCheckpoint(
    'best_model.keras', monitor='val_loss', save_best_only=True
)

history = model.fit(
    X_train, y_train,
    epochs=6,
    batch_size=256,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, checkpoint]
)

KeyboardInterrupt: 

In [2]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np

y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int)

print("Macro F1-score:", f1_score(y_val, y_pred, average='macro'))
print("\nAccuracy:", accuracy_score(y_val.flatten(), y_pred.flatten()))
print("\nDetailed Classification Report:")
print(classification_report(y_val, y_pred, target_names=label_cols))

ImportError: initialization failed

In [4]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.show()

NameError: name 'history' is not defined

In [5]:
model.save("lstm_toxic_comment_model.h5")
print("Model saved successfully as lstm_toxic_comment_model.h5")

NameError: name 'model' is not defined

In [ ]:
from nltk.translate.gale_church import align_blocks
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

def predict_toxicity_with_labels(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')

    pred = model.predict(padded)[0]

    output_lines = []
    output_lines.append(f"{'Категория':<20}{'Задетектировано':<25}{'Процент':<10}")
    output_lines.append("-" * 55)
    for label, p in zip(label_cols, pred):
        detected = "Да" if p > 0.5 else "Нет"
        output_lines.append(f"{label:<20}{detected:<25}{p*100:>6.1f}%")

    return "```\n" + "\n".join(output_lines) + "\n```"

interface = gr.Interface(
    flagging_mode="never",
    fn=predict_toxicity_with_labels,
    inputs=gr.Textbox(
        lines=3,
        placeholder="Ваш текст...",
        label="Впишите сюда текст на английском языке"
    ),
    outputs=gr.Markdown(
        label="Результаты прогнозирования (обнаруженные метки и уверенность)"
    ),
    title="Классификация токсичности комментариев (LSTM)",
    submit_btn="Отправить",
    clear_btn="Очистить",
    css="""
    .gradio-container {
        max-width: 700px !important;
        margin: 0 auto !important;
    }
    .container {
        text-align: center !important;
    }
    """
)

interface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c12a619e666a729aa9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
